### Description
This notebook focuses on scraping the SNPedia website using the MediaWiki API to build a dataframe containing the raw text associated with each SNP associated with each gene. The raw texts are then preprocessed as a preliminary cleaning step to build a second dataset. This dataset is reshaped to be able to be able to easily read in by some other packages but the underlying data is the same. Some additional information is added to this dataframe, including whether or not each gene was mapped in some additional resources, such as the KEGG biochemical pathways database, informed by additional files from other repositories.

In [1]:
from collections import defaultdict
import pandas as pd
import re
import os
import sys
import time
import mwclient
import utils

In [2]:
# Get a list of the genes on SNPedia.
site = mwclient.Site('bots.snpedia.com', path="/")
snpedia_gene_names = [page.name for page in site.Categories["Is_a_gene"]]
print("done")

done


In [3]:
# How many genes are on SNPedia?
print(len(snpedia_gene_names))

2161


In [4]:
# What are the first few values in the list of gene names recovered?
print(snpedia_gene_names[:10])

['A4GALT', 'AANAT', 'AARS', 'AARS2', 'ABCA1', 'ABCA12', 'ABCA3', 'ABCA4', 'ABCA7', 'ABCB1']


In [5]:
# Read in a file that has the list of genes that are mentioned in the KEGG biochemical pathways database.
# This file was built using the KEGG REST API, would have to rerun the pipeline to produces it to get current results.
kegg_df = pd.read_csv(os.path.join("data","kegg_hsa_pathways.csv"))
kegg_gene_identifiers = utils.flatten([gene_identifiers.split("|") for gene_identifiers in kegg_df["gene_identifiers"].values])
kegg_gene_identifier_set = set(kegg_gene_identifiers)
print(len(kegg_gene_identifier_set))

32120


In [6]:
# How many of the genes in SNPedia also have an entry in KEGG?
kegg_gene_identifier_set_lowercased = set([g.lower() for g in list(kegg_gene_identifier_set)])
is_in_kegg = {gene:(gene.lower() in kegg_gene_identifier_set_lowercased) for gene in snpedia_gene_names}
print(len(is_in_kegg))
print(sum(is_in_kegg.values()))

2161
1379


In [7]:
# Should we proceed with all of those genes, or just a subset of them? 
# Supposed to be used for testing, not when running the whole pipeline.
# Could also just use the genes that are present in KEGG or some similar criteria.
snpedia_gene_names_to_use = snpedia_gene_names[1:10]
print(len(snpedia_gene_names_to_use))

9


In [8]:
# Do the web scraping step, building a dictionary mapping genes to SNPs to raw text strings.
GENE_NUM_LIMIT = 1500
PAUSE_AFTER = 50
genes_to_snps_to_text = defaultdict(dict)
for i,gene_name in enumerate(snpedia_gene_names_to_use,1):
    genes_to_snps_to_text[gene_name] = utils.gene_to_snp_texts(site, gene_name)
    if i%PAUSE_AFTER == 0:
        time.sleep(10)
        print(i)
    if i%GENE_NUM_LIMIT == 0:
        break
print("completed the web scraping step")

completed the web scraping step


In [9]:
# Producing a dataset in CSV format that shows genes, SNPs, and the text that was cleaned from each page.
row_tuples_for_raw_df = []
row_tuples_for_cleaned_df = []
gene_to_cleaned_texts = defaultdict(list)
for gene in genes_to_snps_to_text.keys():
    for snp,raw_text in genes_to_snps_to_text[gene].items():
        cleaned_text = utils.clean_raw_page_text(raw_text)
        gene_to_cleaned_texts[gene].append(cleaned_text)
        row_tuples_for_raw_df.append((gene, snp, raw_text))
        row_tuples_for_cleaned_df.append((gene, snp, cleaned_text))

In [10]:
# Generate the dataframe and subset to only include SNPs that had some amount of text extracted, and save as CSV file.
raw_df = pd.DataFrame(row_tuples_for_raw_df, columns = ["gene","snp","text"])
raw_df = raw_df[raw_df["text"] != ""]
raw_df.to_csv(os.path.join("data","snps_and_scraped_text.csv"), index=False)
raw_df.head(10)

,gene,snp,text
0,AANAT,Rs28936679,{{Rsnum\n|rsid=28936679\n|Gene=AANAT\n|Chromos...
1,AANAT,Rs3760138,{{Rsnum\n|rsid=3760138\n|Gene=AANAT\n|Chromoso...
2,AANAT,Rs4238989,{{Rsnum\n|rsid=4238989\n|Gene=AANAT\n|Chromoso...
3,AANAT,Rs8150,{{Rsnum\n|rsid=8150\n|Gene=RHBDF2\n|Chromosome...
4,AARS,Rs1064795664,{{Rsnum\n|rsid=1064795664\n|StabilizedOrientat...
5,AARS,Rs143370729,{{Rsnum\n|rsid=143370729\n|Chromosome=16\n|Ori...
6,AARS,Rs267606621,{{Rsnum\n|rsid=267606621\n|Chromosome=16\n|Ori...
7,AARS,Rs387906792,{{Rsnum\n|rsid=387906792\n|Orientation=minus\n...
8,AARS,Rs786205157,{{Rsnum\n|rsid=786205157\n|Chromosome=16\n|Ori...
9,AARS,Rs797044801,{{Rsnum\n|rsid=797044801\n|Chromosome=16\n|Ori...


In [11]:
# Generate the dataframe and subset to only include SNPs that had some amount of text extracted, and save as CSV file.
cleaned_df = pd.DataFrame(row_tuples_for_cleaned_df, columns = ["gene","snp","text"])
cleaned_df = cleaned_df[cleaned_df["text"] != ""]
cleaned_df.to_csv(os.path.join("data","snps_and_cleaned_text.csv"), index=False)
cleaned_df.head(10)

,gene,snp,text
0,AANAT,Rs28936679,"rs28936679, also known as Ala129Thr or A129T (..."
1,AANAT,Rs3760138,Genetic differences in human circadian clock g...
2,AANAT,Rs4238989,Genetic differences in human circadian clock g...
39,ABCA1,Rs1800977,The -14C->T polymorphism rs1800977 of the ABCA...
40,ABCA1,Rs1883025,Apolipoprotein E levels in cerebrospinal fluid...
41,ABCA1,Rs2020927,"rs2297404, rs2230808, and rs2020927 haplotype ..."
42,ABCA1,Rs2066714,Apolipoprotein E levels in cerebrospinal fluid...
43,ABCA1,Rs2066715,Apolipoprotein E levels in cerebrospinal fluid...
46,ABCA1,Rs2230806,"rs2230806, also known as Arg219Lys or R219K, i..."
47,ABCA1,Rs2230808,"rs2297404, rs2230808, and rs2020927 haplotype ..."


In [12]:
# Concatenate all the raw text that was scraped for all the SNPs for each gene, and generate a dataframe for it.
#gene_to_concatenated_cleaned_texts = {g:" ".join(texts).strip() for g,texts in gene_to_cleaned_texts.items()}
#row_tuples = []
#for gene,text in gene_to_concatenated_cleaned_texts.items():
#    row_tuples.append(("hsa",gene,text))

# Generate the dataframe and save as a CSV file.
#df = pd.DataFrame(row_tuples, columns=["species","unique_gene_identifiers","descriptions"])
#df = df[df["descriptions"] != ""]
#df["other_gene_identifiers"] = ""
#df["gene_models"] = ""
#df["annotations"] = ""
#df["sources"] = "SNPedia"
#df = df[["species","unique_gene_identifiers","other_gene_identifiers","gene_models","descriptions","annotations","sources"]]
#df.to_csv(os.path.join("data","genes_and_concatenated_cleaned_texts.csv"), index=False)
#df.head(10)        

In [13]:
# We might not need to scrape for all the gene names in SNPEedia, because we can only use the ones mentioned in KEGG.
#kegg_filename = "/Users/irbraun/phenologs-with-oats/outputs/06_30_2020_h15m05s52_r1082/part_1_kegg_groupings.csv"
#kegg_df = pd.read_csv(kegg_filename)
#kegg_df = kegg_df[kegg_df["species"]=="hsa"]
#kegg_gene_names = flatten([x.split("|") for x in kegg_df["gene_names"].values])
#kegg_gene_names = [g.upper() for g in kegg_gene_names]
#genes_in_snpedia_and_kegg = list(set(kegg_gene_names).intersection(set(snpedia_gene_names)))
#print(len(genes_in_snpedia_and_kegg))